In [1]:
# Install required libraries
%pip install ipykernel
%pip install scipy
%pip install nltk
%pip install gensim
%pip install transformers torch

# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


/Users/eunsoe/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/eunsoe/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

print("Environment setup complete!")

# File paths
filepaths = [
    'organized_blow_molding_basics.csv',
    'Dupont_Table7.csv',
    'organized_pet_bottle_faults.csv',
    'organized_stressesinpebottles.csv'
]

# Function to load and concatenate text data from each CSV file
def load_text_from_csv(filepaths, text_column):
    text_data = []
    for filepath in filepaths:
        df = pd.read_csv(filepath)
        # Check if the column exists
        if text_column in df.columns:
            text_data.extend(df[text_column].dropna().tolist())
        else:
            print(f"Column '{text_column}' not found in {filepath}. Skipping this file.")
    return text_data

# Identify the text column dynamically
def identify_text_column(filepath):
    df = pd.read_csv(filepath)
    print(f"Columns in {filepath}: {df.columns.tolist()}")

# Uncomment to print columns in all files for manual inspection
# for filepath in filepaths:
#     identify_text_column(filepath)

# Specify the column name containing text in your CSVs
# Replace 'text' with the actual name (e.g., 'content', 'description', etc.)
text_column = 'text'  # Update this based on your CSV inspection

# Load and concatenate text data from CSV files
texts = load_text_from_csv(filepaths, text_column)

# If no text data is found, exit the script
if not texts:
    raise ValueError("No text data found. Please check the text_column value.")

# Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

# Preprocess all text data
processed_texts = [preprocess_text(text) for text in texts]

# Word2Vec embeddings
word2vec_model = Word2Vec(sentences=processed_texts, vector_size=100, window=5, min_count=1, workers=4)
print("Word2Vec Embedding for 'example':", word2vec_model.wv['example'])  # Replace 'example' with a word in your dataset

# BERT embeddings using Hugging Face's transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Sentence-level embedding
    return embeddings

# Generate BERT embeddings for each preprocessed text
bert_embeddings = [get_bert_embeddings(" ".join(tokens)) for tokens in processed_texts]

# Display an example
print("BERT Embedding for first document:", bert_embeddings[0].detach().numpy())


[nltk_data] Downloading package punkt to /Users/eunsoe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eunsoe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/eunsoe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Environment setup complete!
Column 'text' not found in Dupont_Table7.csv. Skipping this file.
Column 'text' not found in organized_pet_bottle_faults.csv. Skipping this file.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/eunsoe/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/share/nltk_data'
    - '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
